# Coffee price prediction
The objective is to predict the rating of coffee beans based on their origin, flavour and tasting notes.

In [ ]:
import numpy as np
import pandas as pd
import json
import os

We will define some utilities related to plotting.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"
from pandas.api.types import is_numeric_dtype


def _hist(s: pd.Series, bins=None) -> pd.Series:
    if is_numeric_dtype(s.dtype):
        if bins is not None:
            count, division = np.histogram(s, bins, density=True)
        else:
            count, division = np.histogram(s, density=True)
        index = (division[:-1] + division[1:]) / 2
        return pd.Series(index=index, data=count, name=s.name), division
    else:
        h = s.value_counts() / len(s)
        if bins is not None:
            h = h[bins]
        return h, h.index


def plot_hist(df: pd.DataFrame, col, by=None, bins=None) -> go.Figure:
    hist_all, bins = _hist(df[col], bins)
    if by:
        data = pd.DataFrame({name: _hist(group[col], bins=bins)[0] for name, group in df.groupby(by)})
    else:
        data = hist_all.to_frame("all")
    fig = data.multiply(100).plot.bar(
        barmode="group", labels={"value": "percentage", "index": col} | ({"variable": by} if by else {})
    )
    if by is None:
        fig.update_layout(showlegend=False)
    return fig


def write_fig_to_html(fig, filename: str) -> None:
    with open(f"charts/{filename}", "w") as f:
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

# EDA

## Data processing

In [ ]:
df = pd.read_csv("data/simplified_coffee.csv")
for col in ["name", "roaster", "roast", "loc_country", "origin", "review"]:
    df[col] = df[col].astype("string")

df["review_date"] = pd.to_datetime(df["review_date"])
df = df.rename(columns={"loc_country": "roaster_country", "100g_USD": "price_per_100g", "origin": "country_of_origin"})
df.head()

Let us first check for NaNs.

In [ ]:
df.isna().sum()

The only column with NaNs is the roast. Since there are only 12 missing values, we could just remove these rows. However, since most coffees have the same roast type (as will see later), let us fill with the modal value.

In [ ]:
df["roast"] = df["roast"].fillna(df["roast"].mode().iloc[0])
df.isna().sum()

Let's fix a typo in the roaster country for one coffee.

In [ ]:
df["roaster_country"] = df["roaster_country"].str.replace("New Taiwan", "Taiwan")

Let's replace some strange characters

In [ ]:
replace = {"’s": "'s", "é": "e", "’": "'"}
for k, v in replace.items():
    df["roaster"] = df["roaster"].str.replace(k, v)

### Roaster

Let's first verify that the information about each roaster (in this case only country) is consistent across all coffees from the same roaster.

In [ ]:
def _assert_identical_values(df: pd.DataFrame) -> pd.Series:
    assert (df.iloc[1:, :] == df.iloc[0, :]).all().all()
    return df.iloc[0, :]


roaster_map = df[["roaster", "roaster_country"]].groupby("roaster").apply(_assert_identical_values)["roaster_country"]

### Flavour notes
As it stands, we cannot glean any information from the review column as it is unstructured. Let's begin by analysing the keywords present in the reviews.

In [ ]:
df["review"]

In [ ]:
from wordcloud import WordCloud, STOPWORDS

COFFEE_WORDS = {"cup", "notes", "finish", "aroma", "hint", "undertones", "mouthfeel", "structure", "toned"}

word_cloud = WordCloud(
    collocations=False, width=2000, height=1000, background_color='white', stopwords=set(STOPWORDS) | COFFEE_WORDS
).generate(" ".join(df["review"]))
word_cloud.to_file('charts/word_cloud.png')
px.imshow(word_cloud)

We can see that the most common words relate to the flavour of the coffee. This suggests that we can extract some features for the different flavours in the coffee.

Using this information and the [coffee flavour wheel](https://www.anychart.com/products/anychart/gallery/Sunburst_Charts/Coffee_Flavour_Wheel.php), we can manually define some flavours and corresponding keywords which are stored in `flavours.json`.

In [ ]:
with open("data/flavours.json", "r") as f:
    FLAVOURS = json.load(f)

We can now add boolean features for each flavour.

In [ ]:
import re


def extract_words(string: str) -> list[str]:
    return re.findall(r'\w+', string.lower())


def rating_contains_words(review: str, keywords: list[str]) -> bool:
    words = extract_words(review)
    for w in keywords:
        if w in words:
            return True
    return False


for flavour, keywords in FLAVOURS.items():
    df[flavour] = df["review"].apply(rating_contains_words, args=(keywords,))

Let's now combine the flavours into a single column.

In [ ]:
df["flavours"] = df.apply(lambda coffee: [flavour for flavour in FLAVOURS if coffee[flavour]], axis=1)

It is also convenient to check how many flavours the different coffees have. If we have done a good job at defining the flavour keywords, we would expect not many coffees to have no flavours.

This appears to be the case. In fact, most coffees have 6 flavours!

In [ ]:
num_flavours = df[list(FLAVOURS.keys())].sum(axis=1)
fig = plot_hist(num_flavours.to_frame("num_flavours"), "num_flavours", bins=[i + 0.5 for i in range(0, 12)])
write_fig_to_html(fig, "num_flavours_hist.html")
fig.show()

### Region of origin
The different regions of the world typically produce coffees which are similar in style. Eg African coffees are typically more acidic. Therefore it seems possible that the region may provide as much information as the country of origin. We will therefore engineer this feature.

In [ ]:
with open("data/regions.json", "r") as f:
    REGIONS = json.load(f)

regions = {}
for r, countries in REGIONS.items():
    for c in countries:
        regions[c] = r

In [ ]:
df["region_of_origin"] = df["country_of_origin"].map(regions).fillna("Other").astype("string")

## Distribution of each feature

### Rating
We can see that the ratings appear to be approximately normally distributed. However, there is a large offset, with the median rating is ~94% which is very high.

In [ ]:
fig = plot_hist(df, "rating", bins=[i + 0.5 for i in range(80, 101)])
write_fig_to_html(fig, "rating_hist.html")
fig.show()

### Price
The distributon for the price of the coffee has a very long tail. This suggests that there may be benefit in applying the log transformation.

In [ ]:
fig = plot_hist(df, "price_per_100g", bins=80)
fig.show()
write_fig_to_html(fig, "price_hist.html")

Now that we have applied the log transformation, the distribution is closer to a normal distribution.

In [ ]:
df["log_price_per_100g"] = df["price_per_100g"].apply(np.log1p)
fig = plot_hist(df, "log_price_per_100g", bins=30)
fig.update_layout(showlegend=False)
fig.show()
write_fig_to_html(fig, "price_log_hist.html")

### Roasting style
The vast majority of the coffee have the medium-light roast type. This large uneveness in the dataset may make it challenging for a model to detect any impact of roast style on coffee rating.

In [ ]:
df["roast"].value_counts()

In [ ]:
fig = plot_hist(df, "roast", bins=["Light", "Medium-Light", "Medium", "Medium-Dark", "Dark"])
fig.show()
write_fig_to_html(fig, "roast_hist.html")

### Roaster country
Most of the data we have is from US rosters.

In [ ]:
df["roaster_country"].value_counts()

If we look at the distribution of pricing for the most common countries, we see that the distribution is quite different in each country. In particular, the coffees sold in the US are much more "peaky". This likely indicates that there is some bias in the dataset. Given that the source of the data is from the US, there are most coffees in the database at an afforable pricepoint (for US customers).

In [ ]:
import plotly.express as px

countries = ["United States", "Taiwan", "Guatemala"]
fig = plot_hist(
    df[df["roaster_country"].apply(lambda c: c in countries)], "price_per_100g", by="roaster_country", bins=80
)
fig.show()
write_fig_to_html(fig, "roaster_country_hist.html")

### Country of origin
As expected, most of the coffees come from the largest coffee producers in the world. All examples are from one of the following regions:

- Africa
- Central or South America
with the exception of Hawaii.

In [ ]:
fig = plot_hist(df, "country_of_origin")
fig.show()
write_fig_to_html(fig, "origin_country_hist.html")

### Region of origin
The vast majority of coffees in the dataset come from the major coffee producing regions of the world as expected.

In [ ]:
fig = plot_hist(df, "region_of_origin", bins=list(REGIONS) + ["Other"])
fig.show()
write_fig_to_html(fig, "origin_region_hist.html")

### Flavour notes
It is useful to examine the popularity of the different flavours, by plotting the histogram. We can see that the most common flavours are:
- Caramelly
- Acidic
- Fruity
- Chocolate

Intuitively, this makes sense as these are the sorts of flavours we see on coffee packets.

In [ ]:
fig = (
    df[list(FLAVOURS.keys())]
    .mean()
    .sort_values(ascending=False)
    .multiply(100)
    .plot.bar(labels={"index": "flavour", "value": "percent"})
)
fig.update_layout(showlegend=False)
fig.show()
write_fig_to_html(fig, "flavour_hist.html")

## Influence on rating

### Price

In [ ]:
fig = df.plot.scatter(x="price_per_100g", y="rating")
fig.show()
write_fig_to_html(fig, "rating_against_price.html")

### Roaster

If we look at the highest and lowest rates coffees, we see that they are dominated by certain roasters. This suggests that either:
- Certain roaster find the best/worst coffees or roast them particulraly well
- The reviewers favour/dislike certainer roasters

In either case, our model may need to access the roaster.

In [ ]:
df.loc[df["rating"] > 96, ["name", "roaster"]].groupby("roaster").count()

In [ ]:
df.loc[df["rating"] < 90, ["name", "roaster"]].groupby("roaster").count()

In [ ]:
data = df.groupby("popular_roaster")["rating"].mean()
fig = data[popular_roasters].plot.bar(labels={"popular_roaster": "roaster", "value": "rating"})
fig.add_hline(df["rating"].mean(), line_dash="dash")
fig.update_yaxes(range=[85, 100])
fig.update_layout(showlegend=False)
fig.show()
write_fig_to_html(fig, "mean_rating_by_roaster.html")

### Roasting style

In [ ]:
fig = plot_hist(df, "rating", by="roast", bins=[i + 0.5 for i in range(80, 101)])
write_fig_to_html(fig, "rating_hist_by_roast.html")
fig.show()

In [ ]:
fig = (
    df.groupby("roast")["rating"]
    .mean()[["Light", "Medium-Light", "Medium", "Medium-Dark", "Dark"]]
    .plot.bar(labels={"value": "rating"})
)
fig.add_hline(df["rating"].mean(), line_dash="dash")
fig.update_yaxes(range=[90, 96])
fig.update_layout(showlegend=False)
fig.show()
write_fig_to_html(fig, "mean_rating_by_roast.html")

### Origin

In [ ]:
fig = plot_hist(df, "rating", by="region_of_origin", bins=[i + 0.5 for i in range(80, 101)])
write_fig_to_html(fig, "rating_hist_by_origin.html")
fig.show()

In [ ]:
fig = df.groupby("region_of_origin")["rating"].mean().plot.bar(labels={"value": "rating"})
fig.add_hline(df["rating"].mean(), line_dash="dash")
fig.update_yaxes(range=[90, 96])
fig.update_layout(showlegend=False)
fig.show()
write_fig_to_html(fig, "mean_rating_by_origin.html")

### Flavour

In [ ]:
rating_with_flavour = pd.Series({f: df.loc[df[f], "rating"].mean() for f in FLAVOURS})
rating_without_flavour = pd.Series({f: df.loc[~df[f], "rating"].mean() for f in FLAVOURS})
fig = (rating_with_flavour - rating_without_flavour).plot.bar(labels={"index": "flavour", "value": "rating_delta"})
fig.add_hline(df["rating"].mean(), line_dash="dash")
fig.update_yaxes(range=[-2, 2])
fig.update_layout(showlegend=False)
fig.show()
write_fig_to_html(fig, "mean_rating_by_flavour.html")

# Modelling

## Feature selection
We can get a quick insight into which features might be important by analysing:
- The correlation coefficient for continuous variables
- The mutual information for categorical variables

### Mutual information

In [ ]:
from sklearn.metrics import mutual_info_score

mutual_info = pd.Series(
    {
        k: mutual_info_score(df[k], df["rating"])
        for k in ["roaster", "roast", "roaster_country", "country_of_origin", "region_of_origin"]
    }
)
mutual_info.sort_values(ascending=False)

In [ ]:
mutual_info_flavours = pd.Series({k: mutual_info_score(df[k], df["rating"]) for k in FLAVOURS})
mutual_info_flavours.sort_values(ascending=False)

### Correlation

In [ ]:
df[["price_per_100g"]].apply(np.log1p).corrwith(df["rating"])

In [ ]:
roasters = df["roaster"].value_counts()
popular_roasters = sorted(roasters[roasters > 10].index)

## Feature engineering

There is evidence that certain roasters product particularly good/poor coffee (or are preferred/disliked by the reviewers). The model may therefore need a feature giving it this information.

We cannot simply convert the roaster using one-hot encoding as there are too many different values. Let us instead only include the most common roasters (those with > 10 coffees).

Let's save this information to file for later use.

In [ ]:
roaster_info = {"known_roasters": roaster_map.to_dict(), "popular_roasters": popular_roasters}
with open("data/roasters.json", "w") as f:
    json.dump(roaster_info, f, indent=4)

In [ ]:
df["popular_roaster"] = df["roaster"].where(df["roaster"].apply(lambda r: r in popular_roasters), "Other")

## Validation framework

In [ ]:
X = df[["popular_roaster", "roaster_country", "roast", "country_of_origin", "price_per_100g"]].copy()
X["price_per_100g"] = X["price_per_100g"].apply(np.log1p)
X["flavours"] = df.apply(
    lambda coffee: [flavour for flavour in ["fruity", "resinous", "spicy", "nutty"] if coffee[flavour]], axis=1
)
y = df["rating"]


* Split the dataset into train/validation/test sets with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test = train_test_split(X, test_size=0.2, random_state=1)
y_train_val, y_test = train_test_split(y, test_size=0.2, random_state=1)

X_train, X_val = train_test_split(X_train_val, test_size=0.25, random_state=1)
y_train, y_val = train_test_split(y_train_val, test_size=0.25, random_state=1)

In [ ]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
dv.fit(X_train.to_dict(orient="records"))


def _transform(df: pd.DataFrame):
    return dv.transform(df.to_dict(orient="records"))

## Linear regression
Let's start with the simplest model which is a linear regressor. 

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

scores_linear = pd.DataFrame(columns=["train", "validation"])
for alpha in [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 100.0]:
    model = Ridge(alpha=alpha)
    model.fit(_transform(X_train), y_train)
    scores_linear.loc[alpha, :] = pd.Series(
        {
            "train": mean_squared_error(y_train, model.predict(_transform(X_train)), squared=False),
            "validation": mean_squared_error(y_val, model.predict(_transform(X_val)), squared=False),
        }
    )

fig = scores_linear.plot(log_x=True, labels={"index": "alpha", "value": "rmse"})
fig.show()
write_fig_to_html(fig, "linear_losses.html")

This suggests that the best value is 1.0 since this gives the same loss on the validation and test sets.

In [ ]:
linear_model = Ridge(alpha=1.0)
linear_model.fit(_transform(X_train_val), y_train_val)

In [ ]:
import plotly.graph_objects as go

fig = px.scatter(x=linear_model.predict(_transform(X_val)), y=y_val)
fig.add_trace(go.Scatter(x=[80, 100], y=[80, 100], showlegend=False))

This model captures the central part of the distribution quite well, but fails to predict the very high or low ratings.

In [ ]:
def plot_model_hist(models: dict[str, Ridge], X: pd.DataFrame, y: pd.Series) -> go.Figure:
    data = pd.concat(
        [pd.DataFrame({"rating": y, "type": "truth"})]
        + [
            pd.DataFrame({"rating": np.round(model.predict(_transform(X)), decimals=0), "type": name})
            for name, model in models.items()
        ]
    )
    return plot_hist(data, "rating", by="type", bins=80)


plot_model_hist({"ridge": linear_model}, X_train_val, y_train_val)
fig.show()
write_fig_to_html(fig, "linear_hist.html")

We can get a bit more insight by evaluating the importance of the difference features.

The price is the biggest influence by far which suggests that either:
- Price is genuinely an indicator of quality
- Price biases the reviewers

Other than the price, the region of origin plays a big influence. Surprisingly only certain flavour notes have much influence; "resinous" and "fruity".

In [ ]:
from sklearn.inspection import permutation_importance

r = permutation_importance(linear_model, _transform(X_train_val), y_train_val, n_repeats=10, random_state=0)
linear_importances = pd.Series(dict(zip(dv.get_feature_names_out(), r.importances_mean)))
linear_importances[linear_importances.abs().sort_values(ascending=False).index].iloc[:10]

## Gradient-boosted trees

In [ ]:
import xgboost as xgb

eval_sets = {
    "train": (_transform(X_train), y_train),
    "validation": (_transform(X_val), y_val),
}

scores_max_depth = {}
for max_depth in [1, 2, 3, 4, 5]:
    xgb_params = {
        'max_depth': max_depth,
        'min_child_weight': 1,
        'objective': 'reg:squarederror',
        'seed': 1,
        'verbosity': 1,
    }

    model = xgb.XGBRegressor(**xgb_params, eval_metric="rmse")
    model.fit(_transform(X_train), y_train, eval_set=list(eval_sets.values()))

    results = model.evals_result()
    scores_max_depth[max_depth] = pd.DataFrame({k: results[f"validation_{i}"]["rmse"] for i, k in enumerate(eval_sets)})

In [ ]:
COLORS = px.colors.qualitative.Plotly
fig = go.Figure()
for i, (depth, scores) in enumerate(scores_max_depth.items()):
    fig.add_trace(
        go.Scatter(x=scores.index, y=scores["train"], name=f"{depth} (train)", line_dash="dash", line_color=COLORS[i])
    )
    fig.add_trace(go.Scatter(x=scores.index, y=scores["validation"], name=f"{depth} (val)", line_color=COLORS[i]))

fig.update_layout(xaxis_title="n_estimators", yaxis_title="rmse", legend_title_text="max_depth")
fig.show()
write_fig_to_html(fig, "trees_losses_depth.html")

Let us select max depth 2 with 20 estimators, since this gives the lowest validation loss without overfitting.

In [ ]:
scores_eta = {}
for eta in [0.01, 0.03, 0.1, 0.3, 1.0]:
    xgb_params = {
        'max_depth': 2,
        'n_estimators': 20,
        "eta": eta,
        'min_child_weight': 1,
        'objective': 'reg:squarederror',
        'seed': 1,
        'verbosity': 1,
    }

    model = xgb.XGBRegressor(**xgb_params, eval_metric="rmse")
    model.fit(_transform(X_train), y_train, eval_set=list(eval_sets.values()))

    results = model.evals_result()
    scores_eta[eta] = pd.DataFrame({k: results[f"validation_{i}"]["rmse"] for i, k in enumerate(eval_sets)})

In [ ]:
fig = go.Figure()
for i, (eta, scores) in enumerate(scores_eta.items()):
    fig.add_trace(
        go.Scatter(x=scores.index, y=scores["train"], name=f"{eta} (train)", line_dash="dash", line_color=COLORS[i])
    )
    fig.add_trace(go.Scatter(x=scores.index, y=scores["validation"], name=f"{eta} (val)", line_color=COLORS[i]))

fig.update_layout(xaxis_title="n_estimators", yaxis_title="rmse", legend_title_text="eta")
fig.show()
write_fig_to_html(fig, "trees_losses_eta.html")

We select `eta` = 0.3.

In [ ]:
xgb_params = {
    'max_depth': 2,
    'n_estimators': 20,
    "eta": 0.3,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'seed': 1,
    'verbosity': 1,
}
xgb_model = xgb.XGBRegressor(**xgb_params, eval_metric="rmse")
xgb_model.fit(_transform(X_train_val), y_train_val, eval_set=[(_transform(X_train_val), y_train_val)])
results = xgb_model.evals_result()
scores_xgb = pd.Series(results[f"validation_0"]["rmse"])

In [ ]:
scores_xgb.plot(labels={"index": "n_estimators", "value": "rmse"})

In the same way as the linear model, this model fails to capture the very low or high ratings.

In [ ]:
plot_model_hist({"xgb": xgb_model}, X_train_val, y_train_val)
write_fig_to_html(fig, "trees_hist.html")
fig.show()

We can see that the biggest influencer is the price as we found with the linear model.

In [ ]:
r = permutation_importance(xgb_model, _transform(X_train_val), y_train_val, n_repeats=10, random_state=0)
xgb_importances = pd.Series(dict(zip(dv.get_feature_names_out(), r.importances_mean)))
xgb_importances[xgb_importances.abs().sort_values(ascending=False).index].iloc[:10]

## Comparison of the models



In [ ]:
models = {"linear": linear_model, "xgb": xgb_model}

Both models perform similarly well on the test set.

In [ ]:
scores_comparison = pd.DataFrame(dtype=float)
for name, model in models.items():
    loss_train = mean_squared_error(y_train_val, model.predict(_transform(X_train_val)), squared=False)
    loss_test = mean_squared_error(y_test, model.predict(_transform(X_test)), squared=False)
    scores_comparison[name] = pd.Series({"train": loss_train, "test": loss_test})

fig = px.bar(scores_comparison.transpose(), barmode="group", labels={"index": "model", "value": "rmse"})
fig.show()
write_fig_to_html(fig, "comparison_losses.html")

We also see that they lead to the same distribution of ratings. This suggests that the model is not the reason for failing to predict the highest/lowest scores is more due to some other more systematic error such as:
- Lack of information in the features (eg perhaps we need more detailed information about the origin)
- System error in the reviews (eg different reviewers)

In [ ]:
fig = plot_model_hist(models, X_test, y_test)
fig.show()
write_fig_to_html(fig, "comparison_hist.html")

## Final model selection
Overall, the two models have very similar performance. Since the linear regression model is simpler (and has slightly better performance), this is the preferred model.